In [1]:
import pixiedust
# pixiedust.installPackage("org.apache.bahir:spark-sql-cloudant_2.11:0")

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200215101153-0000
KERNEL_ID = 4bc2408a-3beb-4ed1-824f-c8133d553bd8
Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
### TODO Please provide your Cloudant credentials in this cell
def readDataFrameFromCloudant(database):

  cloudantdata = spark.read.format("org.apache.bahir.cloudant")\
    .option("cloudant.host",'75e667d0-b6b3-433c-a7b8-3fe72b074604-bluemix.cloudantnosqldb.appdomain.cloud')\
    .option("cloudant.username", '75e667d0-b6b3-433c-a7b8-3fe72b074604-bluemix')\
    .option("cloudant.password",'49dd743301fbd8cdcc1bf39522a10999d4875c288974e0b3e24ddee09b94fee9')\
    .load(database)

  return cloudantdata

In [5]:
df = readDataFrameFromCloudant('training')
df.createOrReplaceTempView('df')

In [6]:
spark.sql('select class, count(class) from df group by class').show()

+-----+------------+
|class|count(class)|
+-----+------------+
|    0|          35|
|    1|          35|
+-----+------------+



In [ ]:
display(df)

_id,_rev,a,ax,ay,az,class,lax,lay,laz,time
31c0c9e35601abfe6c124d4212c358d6,1-a478679d87f06e1cae853efb73e1cf5f,9.854389900330567,0.4884171485900879,0.0,9.873687744140625,0,0.01362034771591425,-0.01362034771591425,0.09534243494272232,1581107283916
31c0c9e35601abfe6c124d4212c372a5,1-341561fc89b0160f5b067ae3ebc0e15e,9.91062541160026,0.40222588181495667,0.028730420395731926,9.90241813659668,0,0.01362034771591425,-0.0272406954318285,0.08172208815813065,1581107305850
31c0c9e35601abfe6c124d4212c397f5,1-0f66b7bd10fa98777a3e0afe16689603,9.84211558323475,0.363918662071228,-0.009576806798577309,9.835380554199219,0,0.0272406954318285,-0.0272406954318285,0.108962781727314,1581107342035
31c0c9e35601abfe6c124d4212c3c7ca,1-c74033085bb3f2b4404b1e311ced26f6,11.802239869685128,11.750741958618164,1.4173674583435059,3.8594532012939453,1,12.489858627319336,-0.3677493631839752,2.71044921875,1581107453621
31c0c9e35601abfe6c124d4212c3e7ae,1-e09e71eb6d6ab32b8c880a79873de710,9.688837010188955,-2.1068975925445557,5.774814605712891,7.489062786102295,1,0.14982382953166962,0.1634441763162613,-0.09534243494272232,1581107469791
322158c6e793f3c3d21471cfc6b97efc,1-c1e95c8e820366e86cbb9d4120ad9bb6,9.88351046351253,0.4405331015586853,0.0,9.873687744140625,0,0.0,-0.01362034771591425,0.08172208815813065,1581107281352
322158c6e793f3c3d21471cfc6b987b4,1-b215fcb845e33519536b543422cffdd9,9.844002428272125,0.4118026793003082,0.009576806798577309,9.835380554199219,0,0.01362034771591425,0.0,0.0272406954318285,1581107285217
322158c6e793f3c3d21471cfc6b9a305,1-b1efa20ee4a7a09e785aa08d06718554,9.8935134640347,0.45010992884635925,0.009576806798577309,9.883264541625977,0,0.054481390863657,-0.01362034771591425,0.08172208815813065,1581107292923
322158c6e793f3c3d21471cfc6b9ad94,1-0228883af0975cd0ae2db6043b2458cc,9.881881756225535,0.40222588181495667,-0.009576806798577309,9.873687744140625,0,0.01362034771591425,-0.01362034771591425,0.04086104407906532,1581107342036
322158c6e793f3c3d21471cfc6b9c8ff,1-8e8840d95e14c364e351d267b41cc96c,5.929969356160485,3.112462282180786,-1.37906014919281,4.855441093444824,1,8.785124778747559,-3.0645782947540283,-0.29964765906333923,1581107418978


In [8]:
# Imports for modelling
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [9]:
# create binary classifier model
vectorAssembler = VectorAssembler(inputCols=["a","ax","ay","az","lax","lay","laz"], outputCol="features")
lr = LogisticRegression(maxIter=1000).setLabelCol("class")
pipeline = Pipeline(stages=[vectorAssembler, lr ])
model = pipeline.fit(df)
result = model.transform(df)

In [10]:
model.stages[1].coefficients

DenseVector([8.8387, 6.7771, 3.5814, -15.6957, -7.2672, -15.8574, 4.6685])

In [11]:
model.stages[1].intercept

45.6523323008108

In [12]:
#evaluate classification accuracy (1.0 = 100% accurate)
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("class")
binEval.evaluate(result)

1.0

In [19]:
# verify model accuracy
new_df = readDataFrameFromCloudant('training')
new_df.createOrReplaceTempView('new_df')
result = model.transform(new_df)
result.createOrReplaceTempView('result')
spark.sql("select a, ax, ay, az, lax, lay, laz, class, prediction from result").show(50)

+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+
|                 a|                 ax|                  ay|                  az|                 lax|                 lay|                 laz|class|prediction|
+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+
| 9.854389900330567| 0.4884171485900879|                 0.0|   9.873687744140625| 0.01362034771591425|-0.01362034771591425| 0.09534243494272232|    0|       0.0|
|  9.91062541160026|0.40222588181495667|0.028730420395731926|    9.90241813659668| 0.01362034771591425| -0.0272406954318285| 0.08172208815813065|    0|       0.0|
|  9.84211558323475|  0.363918662071228|-0.00957680679857...|   9.835380554199219|  0.0272406954318285| -0.0272406954318285|   0.108962781727314|    0|       0.0|
|11.802239869685128| 1